<a href="https://colab.research.google.com/github/iltoum/DataAnalysisPython/blob/master/DIPLWMATIKI_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONTENT BASED

In [ ]:
import pandas as pd
!pip install rake-nltk
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reuters.csv to reuters.csv


In [ ]:
import io
df_reuters = pd.read_csv(io.BytesIO(uploaded['reuters.csv']))

In [ ]:
df_reuters.iloc[0]

web-scraper-order                                          1606153665-2067
web-scraper-start-url    https://www.reuters.com/news/archive/worldNews...
pagination                                                         Earlier
pagination-href          https://www.reuters.com/news/archive/worldNews...
link                     Armenia, Azerbaijan agree to defuse Nagorno-Ka...
link-href                https://www.reuters.com/article/us-armenia-aze...
Title                    Armenia, Azerbaijan agree to defuse Nagorno-Ka...
Date                                                      October 30, 2020
Headline                 GENEVA (Reuters) - The foreign ministers of Ar...
Name: 0, dtype: object

In [ ]:
df_reuters = df_reuters[['Title','Headline']]
df_reuters.head(5)

,Title,Headline
0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",GENEVA (Reuters) - The foreign ministers of Ar...
1,Dealing with Trump presidency nemesis Iran won...,WASHINGTON (Reuters) - When reality TV star Do...
2,Exclusive: Russian hackers targeted California...,WASHINGTON (Reuters) - The group of Russian ha...
3,Spain to send more police to Senegal to curb i...,DAKAR (Reuters) - Spain will increase its poli...
4,Canada expects six million COVID-19 vaccine do...,(Reuters) - Canada expects to receive six mill...


In [ ]:
df_reuters['Headline']=df_reuters['Headline'].apply(str)

In [ ]:
df_reuters['Key_words'] = ""

for index, row in df_reuters.iterrows():
    headline = row['Headline']
    
    r = Rake()

    r.extract_keywords_from_text(headline)

    key_words_dict_scores = r.get_word_degrees()
    
    row['Key_words'] = list(key_words_dict_scores.keys())

df_reuters.drop(columns = ['Headline'], inplace = True)

In [ ]:
df_reuters.set_index('Title', inplace = True)
df_reuters.head(5)

,Key_words
Title,
"Armenia, Azerbaijan agree to defuse Nagorno-Karabakh conflict","[foreign, ministers, killed, nagorno, fighting..."
"Dealing with Trump presidency nemesis Iran won't be 'quick, easy' for Biden","[landmark, deal, aimed, stopping, tehran, isla..."
"Exclusive: Russian hackers targeted California, Indiana Democratic parties","[indiana, email, accounts, knowledge, accordin..."
"Spain to send more police to Senegal to curb illegal migration, says foreign minister","[increase, spain, territory, west, african, co..."
Canada expects six million COVID-19 vaccine doses early in 2021,"[2021, regulatory, approval, parliamentary, co..."


In [ ]:
df_reuters['key_words_string'] = [','.join(map(str, l)) for l in df_reuters['Key_words']]
df_reuters

,Key_words,key_words_string
Title,,
"Armenia, Azerbaijan agree to defuse Nagorno-Karabakh conflict","[foreign, ministers, killed, nagorno, fighting...","foreign,ministers,killed,nagorno,fighting,conf..."
"Dealing with Trump presidency nemesis Iran won't be 'quick, easy' for Biden","[landmark, deal, aimed, stopping, tehran, isla...","landmark,deal,aimed,stopping,tehran,islamic,re..."
"Exclusive: Russian hackers targeted California, Indiana Democratic parties","[indiana, email, accounts, knowledge, accordin...","indiana,email,accounts,knowledge,according,cal..."
"Spain to send more police to Senegal to curb illegal migration, says foreign minister","[increase, spain, territory, west, african, co...","increase,spain,territory,west,african,coast,da..."
Canada expects six million COVID-19 vaccine doses early in 2021,"[2021, regulatory, approval, parliamentary, co...","2021,regulatory,approval,parliamentary,committ..."
...,...,...
Mainland China reports 17 new COVID-19 cases vs. 12 a day earlier,"[health, authority, said, country, ’, mainland...","health,authority,said,country,’,mainland,china..."
U.S. accuses eight in alleged plot to harass and intimidate citizens to return to China,"[criminal, complaint, unsealed, wednesday, chi...","criminal,complaint,unsealed,wednesday,chinese,..."
"Ethiopia sends army into Tigray region, heavy fighting reported","[addis, ababa, heavy, fighting, flared, attack...","addis,ababa,heavy,fighting,flared,attack,feder..."


In [ ]:
df_reuters2 = df_reuters.drop('Key_words', 1)
df_reuters2.head(5)

,key_words_string
Title,
"Armenia, Azerbaijan agree to defuse Nagorno-Karabakh conflict","foreign,ministers,killed,nagorno,fighting,conf..."
"Dealing with Trump presidency nemesis Iran won't be 'quick, easy' for Biden","landmark,deal,aimed,stopping,tehran,islamic,re..."
"Exclusive: Russian hackers targeted California, Indiana Democratic parties","indiana,email,accounts,knowledge,according,cal..."
"Spain to send more police to Senegal to curb illegal migration, says foreign minister","increase,spain,territory,west,african,coast,da..."
Canada expects six million COVID-19 vaccine doses early in 2021,"2021,regulatory,approval,parliamentary,committ..."


In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df_reuters['key_words_string'])


indices = pd.Series(df_reuters.index)
indices[:5]

0    Armenia, Azerbaijan agree to defuse Nagorno-Ka...
1    Dealing with Trump presidency nemesis Iran won...
2    Exclusive: Russian hackers targeted California...
3    Spain to send more police to Senegal to curb i...
4    Canada expects six million COVID-19 vaccine do...
Name: Title, dtype: object

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.03806935, 0.03872015, ..., 0.13043478, 0.04089304,
        0.09567297],
       [0.03806935, 1.        , 0.06780635, ..., 0.03806935, 0.03580574,
        0.04188539],
       [0.03872015, 0.06780635, 1.        , ..., 0.03872015, 0.03641785,
        0.04260143],
       ...,
       [0.13043478, 0.03806935, 0.03872015, ..., 1.        , 0.08178608,
        0.09567297],
       [0.04089304, 0.03580574, 0.03641785, ..., 0.08178608, 1.        ,
        0.04499213],
       [0.09567297, 0.04188539, 0.04260143, ..., 0.09567297, 0.04499213,
        1.        ]])

In [ ]:
def recommendations(Title, cosine_sim = cosine_sim):
    
    recommended_movies = pd.DataFrame()
    
    idx = indices[indices == Title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
  
    top_10_indexes = list(score_series.index)
    
    for i in top_10_indexes:
        recommended_movies=recommended_movies.append([[df_reuters2.index[i],score_series.loc[i]]])
        
    return recommended_movies

In [ ]:
content=recommendations('Armenia, Azerbaijan agree to defuse Nagorno-Karabakh conflict')

In [ ]:
content.head(10)

,0,1
0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",1.000000
0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",0.621874
0,Turkey says Azerbaijan achieved 'sacred succes...,0.430528
0,"Armenia, Azerbaijan, Russia say sign deal to e...",0.404577
0,Russia says discussing U.N. presence in Nagorn...,0.383065
0,Azerbaijan's president says deal is signed to ...,0.354005
0,Leader of Nagorno-Karabakh says ceasefire with...,0.344031
0,Putin calls for Turkish involvement in Nagorno...,0.327144
0,Turkey says in talks on how to monitor Karabak...,0.321029
0,Russia reinforces border guards in Armenia aft...,0.318511


In [ ]:
content.to_csv('contentrating.csv')

COLLABORATIVE

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ratings2.csv to ratings2.csv


In [ ]:
import io
df_ratings = pd.read_csv(io.BytesIO(uploaded['ratings2.csv']))

In [ ]:
df_ratings.head(5)

,UserID,ArticleID,Ratings
0,1,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",2
1,1,Dealing with Trump presidency nemesis Iran won...,1
2,1,Exclusive: Russian hackers targeted California...,3
3,1,Spain to send more police to Senegal to curb i...,4
4,1,Canada expects six million COVID-19 vaccine do...,5


In [ ]:
df_ratings = df_ratings.sample(50)
print(df_ratings)

     UserID                                          ArticleID  Ratings
124       3  Polish cardinal accused of sexual abuse dies a...        3
130       3  Syrian satirist questioned over 'Macron' flogg...        2
29        1  Veteran Syrian diplomat Mekdad named foreign m...        5
92        2  Factbox: As mediation calls mount, who has lev...        5
119       3  No more bullying': fresh start to U.S.-Mexico ...        2
323       7  Veteran Syrian diplomat Mekdad named foreign m...        3
171       4  Burkina opposition candidate alleges 'massive ...        2
460      10  Japan opens airport coronavirus test lab for d...        4
54        2             France reports 271 new COVID-19 deaths        2
51        2  Exclusive: Russian hackers targeted California...        3
287       6  Australia alleges military carried out unlawfu...        1
319       7  Italy has 48 hours to approve new COVID-19 res...        2
258       6  COVID-19 cases soar in Brazil's largest indige...  

In [ ]:
raters=df_ratings.groupby(['ArticleID'])['Ratings'].count()
print(raters)

ArticleID
Australia alleges military carried out unlawful killings in Afghanistan               3
Burkina opposition candidate alleges 'massive fraud' ahead of Sunday vote             1
COVID-19 cases soar in Brazil's largest indigenous reservation                        3
Czechs mark anniversary of Velvet Revolution amid pandemic                            1
Dealing with Trump presidency nemesis Iran won't be 'quick, easy' for Biden           1
England will need five days of lockdown for each day relaxed at Christmas: adviser    1
Ethiopia's Tigray forces' leader denies regional capital circled                      1
Exclusive: Russian hackers targeted California, Indiana Democratic parties            3
Factbox: As mediation calls mount, who has leverage in Ethiopia?                      2
France reports 271 new COVID-19 deaths                                                2
Gunmen kill at least 11 in attack on Iraqi army post in Baghdad, sources say          1
Italy has 48 hours to 

In [ ]:
data=df_ratings.merge(raters,on='ArticleID')
data['count']=data['Ratings_y']
data['Ratings']= data['Ratings_x']
data.drop(['Ratings_x','Ratings_y'],axis=1,inplace=True)

In [ ]:
data.head(5)

,UserID,ArticleID,count,Ratings
0,3,Polish cardinal accused of sexual abuse dies a...,2,3
1,7,Polish cardinal accused of sexual abuse dies a...,2,2
2,3,Syrian satirist questioned over 'Macron' flogg...,1,2
3,1,Veteran Syrian diplomat Mekdad named foreign m...,2,5
4,7,Veteran Syrian diplomat Mekdad named foreign m...,2,3


In [ ]:
data.shape

(50, 4)

In [ ]:
selfjoined = data.merge(data,on='UserID')

In [ ]:
selfjoined.head(5)

,UserID,ArticleID_x,count_x,Ratings_x,ArticleID_y,count_y,Ratings_y
0,3,Polish cardinal accused of sexual abuse dies a...,2,3,Polish cardinal accused of sexual abuse dies a...,2,3
1,3,Polish cardinal accused of sexual abuse dies a...,2,3,Syrian satirist questioned over 'Macron' flogg...,1,2
2,3,Polish cardinal accused of sexual abuse dies a...,2,3,No more bullying': fresh start to U.S.-Mexico ...,2,2
3,3,Polish cardinal accused of sexual abuse dies a...,2,3,"Pompeo says Europe, U.S. need to work together...",2,1
4,3,Polish cardinal accused of sexual abuse dies a...,2,3,Portugal's president ponders COVID-19 state of...,1,5


In [ ]:
selfjoined = selfjoined[selfjoined['ArticleID_x']<selfjoined['ArticleID_y']]

In [ ]:
import numpy as np
selfjoined['r1r2'] = selfjoined['Ratings_x']*selfjoined['Ratings_y']
selfjoined['r1square'] = np.square(selfjoined['Ratings_x'])
selfjoined['r2square'] = np.square(selfjoined['Ratings_y'])

In [ ]:
selfjoined.head(5)

,UserID,ArticleID_x,count_x,Ratings_x,ArticleID_y,count_y,Ratings_y,r1r2,r1square,r2square
1,3,Polish cardinal accused of sexual abuse dies a...,2,3,Syrian satirist questioned over 'Macron' flogg...,1,2,6,9,4
3,3,Polish cardinal accused of sexual abuse dies a...,2,3,"Pompeo says Europe, U.S. need to work together...",2,1,3,9,1
4,3,Polish cardinal accused of sexual abuse dies a...,2,3,Portugal's president ponders COVID-19 state of...,1,5,15,9,25
10,3,No more bullying': fresh start to U.S.-Mexico ...,2,2,Polish cardinal accused of sexual abuse dies a...,2,3,6,4,9
11,3,No more bullying': fresh start to U.S.-Mexico ...,2,2,Syrian satirist questioned over 'Macron' flogg...,1,2,4,4,4


In [ ]:
aggdata=selfjoined.groupby(['ArticleID_x','ArticleID_y'])['Ratings_x','Ratings_y','r1r2','r1square','r2square','count_x','count_y'].sum()
aggdata.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Ratings_x  ...  count_y
ArticleID_x                                        ArticleID_y                                                    ...         
Australia alleges military carried out unlawful... COVID-19 cases soar in Brazil's largest indigen...          5  ...        6
                                                   Factbox: As mediation calls mount, who has leve...          4  ...        2
                                                   France reports 271 new COVID-19 deaths                      4  ...        2
                                                   Italy has 48 hours to approve new COVID-19 rest...          4  ...        3
                                                   Pakistan minister deletes tweet containing Macr...          4  ...        1

[5 rows x 7 columns]

In [ ]:
n=aggdata.shape[0]

In [ ]:
aggdata['corelation'] = (n*aggdata['r1r2'] - aggdata['Ratings_x']*aggdata['Ratings_y']) / \
np.sqrt((n * aggdata['r1square']  - np.square(aggdata['Ratings_x']))*(n * aggdata['r2square']  - np.square(aggdata['Ratings_y'])))

In [ ]:
aggdata['corelation'] = aggdata['corelation']*n/(n+50)

In [ ]:
aggdata.reset_index(inplace=True)
aggdata.head(5)

,ArticleID_x,ArticleID_y,Ratings_x,Ratings_y,r1r2,r1square,r2square,count_x,count_y,corelation
0,Australia alleges military carried out unlawfu...,COVID-19 cases soar in Brazil's largest indige...,5,5,8,17,17,6,6,0.309532
1,Australia alleges military carried out unlawfu...,"Factbox: As mediation calls mount, who has lev...",4,2,8,16,4,3,2,0.668874
2,Australia alleges military carried out unlawfu...,France reports 271 new COVID-19 deaths,4,4,16,16,16,3,2,0.668874
3,Australia alleges military carried out unlawfu...,Italy has 48 hours to approve new COVID-19 res...,4,4,16,16,16,3,3,0.668874
4,Australia alleges military carried out unlawfu...,Pakistan minister deletes tweet containing Mac...,4,3,12,16,9,3,1,0.668874


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reuters.csv to reuters (1).csv


In [ ]:
import io
df_articles = pd.read_csv(io.BytesIO(uploaded['reuters.csv']))

In [ ]:
df_articles = df_articles[['Title','Headline']]
df_articles.head(5)

,Title,Headline
0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",GENEVA (Reuters) - The foreign ministers of Ar...
1,Dealing with Trump presidency nemesis Iran won...,WASHINGTON (Reuters) - When reality TV star Do...
2,Exclusive: Russian hackers targeted California...,WASHINGTON (Reuters) - The group of Russian ha...
3,Spain to send more police to Senegal to curb i...,DAKAR (Reuters) - Spain will increase its poli...
4,Canada expects six million COVID-19 vaccine do...,(Reuters) - Canada expects to receive six mill...


In [ ]:
df_articles.drop_duplicates(subset=None, keep='first', inplace=True)

In [ ]:
aggdata1=aggdata.merge(df_articles,left_on='ArticleID_x',right_on='Title')
aggdata2=aggdata1.merge(df_articles,left_on='ArticleID_y',right_on='Title')

In [ ]:
aggdata2.head(20)

,ArticleID_x,ArticleID_y,Ratings_x,Ratings_y,r1r2,r1square,r2square,count_x,count_y,corelation,Title_x,Headline_x,Title_y,Headline_y
0,Australia alleges military carried out unlawfu...,COVID-19 cases soar in Brazil's largest indige...,5,5,8,17,17,6,6,0.309532,Australia alleges military carried out unlawfu...,SYDNEY (Reuters) - The head of Australia’s Def...,COVID-19 cases soar in Brazil's largest indige...,BRASILIA (Reuters) - The coronavirus has sprea...
1,Australia alleges military carried out unlawfu...,"Factbox: As mediation calls mount, who has lev...",4,2,8,16,4,3,2,0.668874,Australia alleges military carried out unlawfu...,SYDNEY (Reuters) - The head of Australia’s Def...,"Factbox: As mediation calls mount, who has lev...",NAIROBI (Reuters) - Ethiopia has rebuffed atte...
2,COVID-19 cases soar in Brazil's largest indige...,"Factbox: As mediation calls mount, who has lev...",5,7,22,17,29,6,4,0.662669,COVID-19 cases soar in Brazil's largest indige...,BRASILIA (Reuters) - The coronavirus has sprea...,"Factbox: As mediation calls mount, who has lev...",NAIROBI (Reuters) - Ethiopia has rebuffed atte...
3,Czechs mark anniversary of Velvet Revolution a...,"Factbox: As mediation calls mount, who has lev...",3,5,15,9,25,1,2,0.668874,Czechs mark anniversary of Velvet Revolution a...,PRAGUE (Reuters) - Czechs on Tuesday marked mo...,"Factbox: As mediation calls mount, who has lev...",NAIROBI (Reuters) - Ethiopia has rebuffed atte...
4,Exclusive: Russian hackers targeted California...,"Factbox: As mediation calls mount, who has lev...",3,5,15,9,25,3,2,0.668874,Exclusive: Russian hackers targeted California...,WASHINGTON (Reuters) - The group of Russian ha...,"Factbox: As mediation calls mount, who has lev...",NAIROBI (Reuters) - Ethiopia has rebuffed atte...
5,Australia alleges military carried out unlawfu...,France reports 271 new COVID-19 deaths,4,4,16,16,16,3,2,0.668874,Australia alleges military carried out unlawfu...,SYDNEY (Reuters) - The head of Australia’s Def...,France reports 271 new COVID-19 deaths,PARIS (Reuters) - The number of people to die ...
6,COVID-19 cases soar in Brazil's largest indige...,France reports 271 new COVID-19 deaths,4,2,8,16,4,3,2,0.668874,COVID-19 cases soar in Brazil's largest indige...,BRASILIA (Reuters) - The coronavirus has sprea...,France reports 271 new COVID-19 deaths,PARIS (Reuters) - The number of people to die ...
7,Czechs mark anniversary of Velvet Revolution a...,France reports 271 new COVID-19 deaths,3,2,6,9,4,1,2,0.668874,Czechs mark anniversary of Velvet Revolution a...,PRAGUE (Reuters) - Czechs on Tuesday marked mo...,France reports 271 new COVID-19 deaths,PARIS (Reuters) - The number of people to die ...
8,Exclusive: Russian hackers targeted California...,France reports 271 new COVID-19 deaths,3,2,6,9,4,3,2,0.668874,Exclusive: Russian hackers targeted California...,WASHINGTON (Reuters) - The group of Russian ha...,France reports 271 new COVID-19 deaths,PARIS (Reuters) - The number of people to die ...
9,"Factbox: As mediation calls mount, who has lev...",France reports 271 new COVID-19 deaths,5,2,10,25,4,2,2,0.668874,"Factbox: As mediation calls mount, who has lev...",NAIROBI (Reuters) - Ethiopia has rebuffed atte...,France reports 271 new COVID-19 deaths,PARIS (Reuters) - The number of people to die ...


In [ ]:
def recommendation(Title):
    recommended_movies = []
    data =aggdata2[aggdata2['Title_x']==Title]
    data = data.sort_values(by='corelation',ascending=False)
    return data

In [ ]:
collabaritive=recommendation('England will need five days of lockdown for each day relaxed at Christmas: adviser')[['Title_y','corelation']]
collabaritive['Title'] = collabaritive['Title_y'].str.split(' \(').str.get(0)
collabaritive.drop('Title_y',axis=1,inplace=True)
collabaritive.head(5)

,corelation,Title
40,0.668874,Taiwan hopes for close U.S. cooperation in cal...
49,0.668874,Thank you Tegel: Berliners bid emotional farew...
71,0.668874,Exclusive: Russian hackers targeted California...
80,0.668874,Japan opens airport coronavirus test lab for d...


HYBRID

In [ ]:
contentrating = pd.read_csv('contentrating.csv')
contentrating.head(10)

,Unnamed: 0,0,1
0,0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",1.000000
1,0,"Armenia, Azerbaijan agree to defuse Nagorno-Ka...",0.621874
2,0,Turkey says Azerbaijan achieved 'sacred succes...,0.430528
3,0,"Armenia, Azerbaijan, Russia say sign deal to e...",0.404577
4,0,Russia says discussing U.N. presence in Nagorn...,0.383065
5,0,Azerbaijan's president says deal is signed to ...,0.354005
6,0,Leader of Nagorno-Karabakh says ceasefire with...,0.344031
7,0,Putin calls for Turkish involvement in Nagorno...,0.327144
8,0,Turkey says in talks on how to monitor Karabak...,0.321029
9,0,Russia reinforces border guards in Armenia aft...,0.318511


In [ ]:
hybrid=contentrating.merge(collabaritive,left_on='0',right_on='Title')
hybrid=hybrid[['Title','1','corelation']]
hybrid['wcorelation'] = (hybrid['1'] + hybrid['corelation'])/2

In [ ]:
hybrid.head(5)

,Title,1,corelation,wcorelation
0,Japan opens airport coronavirus test lab for d...,0.093250,0.668874,0.381062
1,Thank you Tegel: Berliners bid emotional farew...,0.045502,0.668874,0.357188
2,Taiwan hopes for close U.S. cooperation in cal...,0.040129,0.668874,0.354501
3,Exclusive: Russian hackers targeted California...,0.038720,0.668874,0.353797


In [ ]:
hybrid.to_csv('hybridsystem.csv')